# **Random Forest**
- Using data that was annotated
- Training it on the unbalanced data
- Using avgpool
- 1 second window

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Standard libraries
import numpy as np
import pandas as pd
import time
import os

# For audio
from IPython.display import Audio
import librosa

# For preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

# For modeling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Operational
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import scipy.ndimage
import pygame
import time
from scipy.signal import butter, filtfilt
import random

pygame 2.6.1 (SDL 2.28.4, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
pkl_path = '/content/drive/My Drive/Final-Year-Project/Dataset/Project-V4/feature-extraction/Annotated/Regular/AveragePooled/split_features_3s_all_1D.pkl'

# Load the pickle file
with open(pkl_path, 'rb') as file:
    data = pickle.load(file)

In [4]:
train_data = data['train'].copy()
val_data = data['val'].copy()

In [5]:
train_data

{'melspectrogram': array([[9.4866091e-06, 9.5487494e-06, 9.6551566e-06, ..., 9.8847534e-07,
         9.3044491e-07, 9.0232516e-07],
        [1.2129851e-05, 1.2414495e-05, 1.2921279e-05, ..., 5.2223759e-06,
         4.9851474e-06, 4.8430848e-06],
        [5.8306232e-05, 5.9333925e-05, 6.1116712e-05, ..., 1.7024886e-06,
         1.6033545e-06, 1.5549887e-06],
        ...,
        [1.7914819e-04, 1.8226101e-04, 1.8773414e-04, ..., 2.3687387e-06,
         1.6498615e-06, 1.5470791e-06],
        [1.9102510e-05, 1.9464049e-05, 2.0094065e-05, ..., 4.9249811e-06,
         4.0005261e-06, 3.6747458e-06],
        [9.0032081e-06, 9.2558430e-06, 9.7203119e-06, ..., 9.5587347e-07,
         4.5187107e-07, 4.2664561e-07]], dtype=float32),
 'mfcc': array([[-1.        , -0.03889354, -0.64160925, ..., -0.02376524,
          0.02349628, -0.00874254],
        [-1.        , -0.0368154 , -0.633421  , ..., -0.02121266,
          0.0290794 , -0.01421679],
        [-1.        , -0.028492  , -0.6135318 , ..., -0.

In [6]:
train_labels = train_data['label'].copy()
temp = train_data.copy()
del temp['label']
tr_features = temp

In [7]:
val_labels = val_data['label'].copy()
temp = val_data.copy()
del temp['label']
v_features = temp

# Shuffle Data

In [8]:
def shuffle_data(input_label, input_features):
  input_len = len(input_label)
  np.random.seed(1826)
  input_indices = np.random.permutation(input_len)
  input_features = {key: np.array([input_features[key][i] for i in input_indices]) for key in input_features} # dictionary comprehension
  input_label = np.array([input_label[i] for i in input_indices])

  return input_label, input_features

In [9]:
train_y, train_features = shuffle_data(train_labels, tr_features)

In [10]:
val_y, val_features = shuffle_data(val_labels, v_features)

In [11]:
display(train_y.shape)
display(train_y[:15])

(12565,)

array([ 0, 19,  2,  1, 11, 17, 15,  2,  2,  2,  0,  1,  1, 12,  0],
      dtype=int32)

In [12]:
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)
  display(train_features[key][0])

'melspectrogram'

(12565, 60)

array([5.0738169e-07, 5.2262533e-07, 5.5045160e-07, 5.9217962e-07,
       6.5191085e-07, 1.0943645e-06, 4.3586356e-06, 1.9036837e-05,
       1.2136879e-04, 2.6661658e-04, 8.2672836e-04, 2.7233763e-03,
       1.0063773e-02, 1.6930899e-02, 2.1348633e-02, 4.7072649e-02,
       7.6561354e-02, 4.7709879e-02, 6.2269606e-02, 7.6247416e-02,
       6.1358120e-02, 3.9835755e-02, 4.4739708e-02, 5.0686374e-02,
       4.4587012e-02, 5.6312285e-02, 4.7983538e-02, 5.3998958e-02,
       9.1156095e-02, 1.2960179e-01, 1.3472772e-01, 7.4508280e-02,
       2.8677055e-01, 4.0667152e-01, 7.1514398e-01, 6.9495946e-01,
       3.1333745e-01, 2.0148984e-01, 1.1326054e-01, 6.4187691e-02,
       7.1493596e-02, 8.1204779e-02, 7.1447775e-02, 5.0326228e-02,
       3.6310278e-02, 2.3586553e-02, 1.3296669e-02, 4.2282213e-03,
       3.2108359e-03, 4.2909747e-03, 3.0864956e-02, 6.0674440e-02,
       5.0374899e-02, 3.0964591e-02, 4.0448662e-03, 3.0534746e-04,
       3.1673382e-05, 3.8426388e-06, 2.3940433e-06, 2.1029000e

'mfcc'

(12565, 20)

...

(12565, 12)

array([0.18776874, 0.15328921, 0.33524922, 0.454852  , 0.7760426 ,
       0.67847323, 0.3407292 , 0.24142714, 0.18064852, 0.15764236,
       0.18976806, 0.2140399 ], dtype=float32)

'rms'

(12565, 1)

array([0.0295004], dtype=float32)

In [13]:
display(val_y.shape)
display(val_y[:15])

(3318,)

array([ 3,  2,  1,  0,  0,  8, 19,  2, 19,  5, 19, 15, 11,  7,  2],
      dtype=int32)

In [14]:
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)
  display(val_features[key][0])

'melspectrogram'

(3318, 60)

array([1.8282815e-04, 1.8516471e-04, 1.8923497e-04, 1.9535325e-04,
       2.0405794e-04, 2.1621672e-04, 2.3355127e-04, 2.6386356e-04,
       3.8294177e-04, 8.7105710e-04, 4.0952233e-03, 7.9782819e-03,
       1.2021556e-02, 1.1450265e-02, 1.8598879e-02, 4.5423362e-02,
       5.1051736e-02, 2.1339525e-02, 2.7392777e-02, 2.3795826e-02,
       1.3348093e-02, 1.3525566e-02, 8.1301248e-03, 8.5273301e-03,
       1.1604097e-02, 7.5979652e-03, 4.1982550e-03, 4.9553681e-03,
       6.7672539e-03, 8.9110127e-03, 1.0491555e-02, 8.9149661e-03,
       8.2060061e-03, 6.9327699e-03, 7.8629563e-03, 8.1582973e-03,
       7.1896906e-03, 3.3999698e-03, 3.0463785e-03, 5.0475681e-03,
       5.0008059e-03, 4.5382837e-03, 4.0109674e-03, 4.0369830e-03,
       9.5070582e-03, 1.8515797e-02, 5.2194532e-02, 2.4449244e-01,
       2.7477503e-01, 5.4956347e-01, 6.2898314e-01, 4.4136560e-01,
       5.3704370e-02, 2.4117595e-02, 1.1598948e-02, 4.3818848e-03,
       2.8213693e-04, 6.7681176e-06, 2.1489736e-06, 2.1393287e

'mfcc'

(3318, 20)

...

(3318, 12)

array([0.02229075, 0.02539484, 0.03112359, 0.1376025 , 0.2878612 ,
       0.39549676, 0.5717799 , 0.5721824 , 0.20036055, 0.06311192,
       0.03563747, 0.02806006], dtype=float32)

'rms'

(3318, 1)

array([0.03920381], dtype=float32)

## **Random Forest Model**

In [15]:
train_results = {}
val_results = {}

### **With all the features**

In [16]:
training_features = np.concatenate((train_features['mfcc'], train_features['chroma'], train_features['rms'], train_features['melspectrogram']), axis=1)

training_features.shape

(12565, 93)

In [17]:
validation_features = np.concatenate((val_features['mfcc'], val_features['chroma'], val_features['rms'], val_features['melspectrogram']), axis=1)

validation_features.shape

(3318, 93)

Fit the model with training data

In [18]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [19]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['all_features'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['all_features'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5910186859553949


In [20]:
train_yhat_result = rf.predict(training_features)

print('Training classification Report \n')
print(classification_report(train_y, train_yhat_result))

Training classification Report 

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2060
           1       1.00      1.00      1.00      2477
           2       1.00      1.00      1.00      2203
           3       1.00      1.00      1.00       437
           4       1.00      1.00      1.00       218
           5       1.00      1.00      1.00       387
           6       1.00      1.00      1.00       167
           7       1.00      1.00      1.00       263
           8       1.00      1.00      1.00       196
           9       1.00      1.00      1.00       443
          10       1.00      1.00      1.00       191
          11       1.00      1.00      1.00       211
          12       1.00      1.00      1.00       378
          13       1.00      1.00      1.00       232
          14       1.00      1.00      1.00       188
          15       1.00      1.00      1.00      1133
          16       1.00      1.00      1.00     

In [21]:
val_yhat_result = rf.predict(validation_features)

print('Validation classification Report \n')
print(classification_report(val_y, val_yhat_result))

Validation classification Report 

              precision    recall  f1-score   support

           0       0.61      0.75      0.67       455
           1       0.37      0.73      0.49       492
           2       0.69      0.31      0.43       889
           3       0.81      0.98      0.89       150
           4       0.65      0.16      0.26        67
           5       0.35      0.67      0.46        43
           6       0.38      0.25      0.30        24
           7       0.71      0.66      0.68        44
           8       0.39      0.68      0.50        50
           9       0.93      0.95      0.94       169
          10       0.63      0.60      0.62        53
          11       0.55      0.27      0.36        66
          12       0.77      0.73      0.75        59
          13       0.48      0.47      0.48        57
          14       0.74      0.61      0.67        38
          15       0.66      0.55      0.60       311
          16       0.98      0.91      0.94   

### **With MFCCs only**

In [22]:
training_features = train_features['mfcc']

training_features.shape

(12565, 20)

In [23]:
validation_features = val_features['mfcc']

validation_features.shape

(3318, 20)

In [24]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [25]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5376732971669681


### **With Mel-Spectrogram Only**

In [26]:
training_features = train_features['melspectrogram']

training_features.shape

(12565, 60)

In [27]:
validation_features = val_features['melspectrogram']

validation_features.shape

(3318, 60)

In [28]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [29]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.4957805907172996


## **With Chroma Only**

In [30]:
training_features = train_features['chroma']

training_features.shape

(12565, 12)

In [31]:
validation_features = val_features['chroma']

validation_features.shape

(3318, 12)

In [32]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [33]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['chroma'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['chroma'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.3649789029535865


## **With MFCCs and Mel-Spectrogram**

In [34]:
training_features = np.concatenate((train_features['mfcc'], train_features['melspectrogram']), axis=1)

training_features.shape

(12565, 80)

In [35]:
validation_features = np.concatenate((val_features['mfcc'], val_features['melspectrogram']), axis=1)

validation_features.shape

(3318, 80)

In [36]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [37]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc_melspectrogram'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc_melspectrogram'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5699216395418927


## **With MFCCs and Chroma**

In [38]:
training_features = np.concatenate((train_features['mfcc'], train_features['chroma']), axis=1)

training_features.shape

(12565, 32)

In [39]:
validation_features = np.concatenate((val_features['mfcc'], val_features['chroma']), axis=1)

validation_features.shape

(3318, 32)

In [40]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [41]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc_chroma'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc_chroma'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5443037974683544


## **With MFCCs and RMS**

In [42]:
training_features = np.concatenate((train_features['mfcc'], train_features['rms']), axis=1)

training_features.shape

(12565, 21)

In [43]:
validation_features = np.concatenate((val_features['mfcc'], val_features['rms']), axis=1)

validation_features.shape

(3318, 21)

In [44]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [45]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc_rms'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc_rms'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5572634116937915


## **With Chroma and RMS**

In [46]:
training_features = np.concatenate((train_features['chroma'], train_features['rms']), axis=1)

training_features.shape

(12565, 13)

In [47]:
validation_features = np.concatenate((val_features['chroma'], val_features['rms']), axis=1)

validation_features.shape

(3318, 13)

In [48]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [49]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['chroma_rms'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['chroma_rms'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.39511754068716093


## **With Mel-Spectrogram and Chroma**

In [50]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['chroma']), axis=1)

training_features.shape

(12565, 72)

In [51]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['chroma']), axis=1)

validation_features.shape

(3318, 72)

In [52]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [53]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_chroma'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_chroma'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5126582278481012


## **With Mel-Spectrogram and RMS**

In [54]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['rms']), axis=1)

training_features.shape

(12565, 61)

In [55]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['rms']), axis=1)

validation_features.shape

(3318, 61)

In [56]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [57]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_rms'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_rms'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5039180229053647


## **With Mel-Spectrogram, MFCCs, RMS**

In [58]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['rms'], train_features['mfcc']), axis=1)

training_features.shape

(12565, 81)

In [59]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['rms'], val_features['mfcc']), axis=1)

validation_features.shape

(3318, 81)

In [60]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [61]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_rms_mfcc'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_rms_mfcc'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.567209162145871


## **With Mel-Spectrogram, MFCCs, Chroma**

In [62]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['chroma'], train_features['mfcc']), axis=1)

training_features.shape

(12565, 92)

In [63]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['chroma'], val_features['mfcc']), axis=1)

validation_features.shape

(3318, 92)

In [64]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [65]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_chroma_mfcc'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_chroma_mfcc'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5855937311633514


# Review results from all models

In [66]:
train_results_df = pd.DataFrame(list(train_results.items()), columns=['Features', 'Train_Accuracy']).round(2)
val_results_df = pd.DataFrame(list(val_results.items()), columns=['Features', 'Val_Accuracy']).round(2)

result_df = train_results_df.merge(val_results_df, on='Features')
result_df = result_df.sort_values('Features')
result_df

,Features,Train_Accuracy,Val_Accuracy
0,all_features,1.0,0.59
3,chroma,1.0,0.36
7,chroma_rms,1.0,0.40
2,melspectrogram,1.0,0.50
8,melspectrogram_chroma,1.0,0.51
11,melspectrogram_chroma_mfcc,1.0,0.59
9,melspectrogram_rms,1.0,0.50
10,melspectrogram_rms_mfcc,1.0,0.57
1,mfcc,1.0,0.54
5,mfcc_chroma,1.0,0.54
